Задание:

> Play with NATS
> 
> Test Pub/Sub
> 
> Test Work Queue (group consumers)
> 
> Test Request/Response

В первом задании я использовал NATS Core, чтобы сделать очередь для микросервисного взаимодействия. Я столкнулся с фундаментальными ограничениями, из-за которых которых пришлось мириться с тем, что:
* Если потушить брокера, то потеряются сообщения, на которые уже запрошен ответ, но ещё не получен.
* Если потушить консьюмера, то вместе с ним пропадут сообщения, взятые им в обработку (включая буфер).
* Если потушить продьюсера, то, в целом, ничего не теряется, но если какие-то сообщения упавшего продьюсера ожидают ответа, то ответ пропадёт. Второй продьюсер не примет на себя ответ, направленный другому.

Сейчас попробую решить ту же задачу с использованием NATS JetStream и кластера из трёх нод. Кластер поднимаю с помощью docker compose. Устанавливаю CLI.

In [29]:
!nats context add sys --server localhost:4222,localhost:4223,localhost:4224 --user admin --password admin
!nats context add user --server localhost:4222,localhost:4223,localhost:4224 --user user --password user
!nats context select user &> /dev/null

NATS Configuration Context "sys"

      Server URLs: localhost:4222,localhost:4223,localhost:4224
         Username: admin
         Password: *********
            Token: admin
             Path: /home/timosha/.config/nats/context/sys.json

NATS Configuration Context "user"

      Server URLs: localhost:4222,localhost:4223,localhost:4224
         Username: user
         Password: *********
            Token: user
             Path: /home/timosha/.config/nats/context/user.json



In [35]:
!nats stream add REQUESTS --storage memory --subjects "users.requests" --replicas 3 --retention work --discard new --max-msgs=-1 --max-msgs-per-subject=-1 --max-bytes=-1 --max-age 5s --max-msg-size=-1 --dupe-window 0 --no-allow-rollup --deny-delete --no-deny-purge

Stream REQUESTS was created

Information for Stream REQUESTS created 2025-07-23 23:50:30

             Subjects: users.requests
             Replicas: 3
              Storage: Memory

Options:

            Retention: WorkQueue
     Acknowledgements: true
       Discard Policy: New
     Duplicate Window: 5s
    Allows Msg Delete: false
         Allows Purge: true
       Allows Rollups: false

Limits:

     Maximum Messages: unlimited
  Maximum Per Subject: unlimited
        Maximum Bytes: unlimited
          Maximum Age: 5.00s
 Maximum Message Size: unlimited
    Maximum Consumers: unlimited


Cluster Information:

                 Name: nats
               Leader: inst-3
              Replica: inst-1, current, seen 0.00s ago
              Replica: inst-2, current, seen 0.00s ago

State:

             Messages: 0
                Bytes: 0 B
             FirstSeq: 0
              LastSeq: 0
     Active Consumers: 0


* `--storage file`: сообщения из очереди хранятся в памяти (с коротким TTL в 5 секунд нецелесообразно тратить время на диск)
* `--replicas 3`: очередь реплицируется трижды
* `--retention work`: сообщения удаляются из стрима, как только они обработаны (acked) всеми, кому доставлены
* `--discard new`: если стрим заполнен до предела, он будет заблокирован для записи
* `--max-msgs -1`: нет ограничения на максимальное кол-во сообщений в стриме
* `--max-msgs-per-subject -1`: нет ограничения на максимальное кол-во сообщений в сабжекте
* `--max-bytes -1`: нет ограничения на размер стрима
* `--max-age 5s`: сообщения удаляются через 5 секунд после публикации
* `--max-msg-size -1`: сообщения не ограничены по размеру
* `--dupe-window 0`: отключает отслеживание дубликатов
* `--no-allow-rollup`: запрещает свёртку сообщений
* `--deny-delete`: запрещает удалять сообщения вручную
* `--no-deny-purge`: позволяет вручную чистить весь стрим целиком

Если ответы отправляются на временные `_INBOX.*` сабжекты, они не сохраняются в стрим. А это не здорово. Ответы тоже хочется хранить персистентно. 
Создаю стрим для ответов. Он будет похож на первый стрим, но без TTL и не более одного сообщения в сабжекте - нужен только один ответ на каждый запрос.

In [37]:
!nats stream add RESPONSES --storage memory --subjects "users.responses.*" --replicas 3 --retention work --discard new --max-msgs=-1 --max-msgs-per-subject=1 --max-bytes=-1 --max-age=-1 --max-msg-size=-1 --dupe-window 0 --no-allow-rollup --deny-delete --no-deny-purge

Stream RESPONSES was created

Information for Stream RESPONSES created 2025-07-23 23:59:13

             Subjects: users.responses.*
             Replicas: 3
              Storage: Memory

Options:

            Retention: WorkQueue
     Acknowledgements: true
       Discard Policy: New
     Duplicate Window: 2m0s
    Allows Msg Delete: false
         Allows Purge: true
       Allows Rollups: false

Limits:

     Maximum Messages: unlimited
  Maximum Per Subject: 1
        Maximum Bytes: unlimited
          Maximum Age: unlimited
 Maximum Message Size: unlimited
    Maximum Consumers: unlimited


Cluster Information:

                 Name: nats
               Leader: inst-3
              Replica: inst-1, current, seen 0.00s ago
              Replica: inst-2, current, seen 0.00s ago

State:

             Messages: 0
                Bytes: 0 B
             FirstSeq: 0
              LastSeq: 0
     Active Consumers: 0


Теперь надо внести изменения в продьюсера и коньюмера. Планирую:
1. Явным образом добавить INBOX сабжект, перехватывать сообщения в сабжекты в JetStream. Аналог `request` надо cделать самому на основе одноразовой подписки на сабжект `reply_to`: дело в том, что питоновский `request` безальтернативно принимает ответы в свой неперсистентный INBOX.
2. Если перезапускается продьюсер, то он сможет получить ответы на предыдущие запросы только если будет помнить, на какие запросы ждал ответа. Так он сможет подписаться на сабжекты, куда эти ответы прийдут. Но мне больше нравится идея связать продьюсеров между собой, чтобы один из них мог передать свои задачи другим (прежде чем завершиться). Это можно сделать через отдельный сабжект, который не обязан быть персистентным. На этот сабжект все живые продьюсеры будут подписаны из-под одной consumer queue.
3. Пусть консьюмеры пуллят задачи из стрима. Это решит проблему, возникшую ранее, когда у нас появился медленный консьюмер. Медленный консьюмер набирал столько же задач, сколько быстрый - и просрочивал выполнение. Пуллить задачу могут только консьюмеры JetStream (стандартная подписка на сабжект из NATS Core не подходит).
4. Важно, чтобы продьюсер мог получить ответ, если задача ему досталась от другого продьюсера. Это значит, что продьюсер может подписаться сабжект уже после того, как в него опубликуют сообщение. В NATS Core так сделать нельзя: если никто не принимает сообщение, оно пропадает. Поэтому здесь тоже нужен консьюмер JetStream.

Разделение между продьюсерами и консьюмерами становится довольно условным - и те, и те компоненты теперь выступают в обеих ролях.

![image.png](./pic/_1.png)

Если по одному консьюмеру и продьюсеру, то всё работает. Добавлю второго консьюмера

![image.png](./pic/_2.png)

1. Благодаря `ack_policy="explicit"` сообщение не считается обработанным, пока консьюмер не сделает `ack`. Первый консьюмер при отключении сделал `nack`, благодаря чему сообщение с `user_id=7` снова оказалось доступно к прочтению из очереди. Его подхватил и обработал второй консьюмер. По логам продьюсера видно, что в конечном итоге тот получил ответ с `user_id=7` именно от второго консьюмера
2. Благодаря pull-модели каждый консьюмер берёт задачи в своём темпе. Быстрый консьюмер набирает себе больше задач, медленный - меньше.
3. Консьюмеры делят между собой сообщения благодаря тому, что работают изнутри одной durable-группы. Когда очередь JetStream создана по модели `WorkQueue` (сообщения живут до первого `ack`-а) и очередь разбирается через pull-консьюмеров, нельзя создать больше одной durable-группы на сабжект. У нас как раз она одна.
4. Если поднять продьюсеров раньше чем консьюмеров, это не помешает консьюмерам получить сообщения - NATS их держит на своей стороне на протяжении всего времени TTL.

Теперь протестирую работу с несколькими продьюсерами.

![image.png](./pic/_3.png)

Второй продьюсер перед отключением смог передать свои задачи первому продьюсеру. Полученные от второго продьюсера задачи первый продьюсер успешно отправил на обработку и дождался ответа на них.

Если продьюсер кидает реквест и падает, то реквест будет подобран консьюмером и ответ запушен в явно указанный топик - консьюмеры о продьюсерах ничего не знают. Продьюсер перед падением передаст свои задачи другим продьюсерам. Другие продьюсеры ещё раз поставят те же задачи и, возможно, консьюмеры ещё раз выполнят ту же работу. Это исключительный случай и это некритично: в INBOX-сабжект нельзя одновременно публикавать больше одного сообщения (настройка стрима), плюс продьюсеры читают из этих сабжектов не более одного сообщения.

Последнее, что хочется протестировать - падение брокера. 

![image.png](./pic/_4.png)

Падение одного из трёх ни на что не повлияло. Первый консьюмер столкнулся с ошибкой, однако библиотека для работы с NATS эту ошибку подавила (делаю такой вывод на основании того, что если бы ошибку обрабатывал написанный мною код, программа бы завершилась). Брокер повторно отдал то сообщение, с которым не получилось поработать у первого консьюмера - это видно по тому, что ответ на `user_id=60` первый продьюсер получил в итоге от второго консьюмера, а не первого.